<a href="https://colab.research.google.com/github/SanjitaPrajapati/AI4CCEE_Fall_2024/blob/main/in_class_activity/lecture8/submissions/sanjita_lecture8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity 1

1. GPS DATA TABLE
2. LRS DATA TABLE

# Activity 2

1. Make point feature from Point 1 : 41.948637 , -93.610065 using ST_POINT

SELECT st_point(-93.610065 ,41.948637) AS point_feature;

2. SELECT st_distance
(
    st_point(-93.610065,41.948637),
    st_point(-93.610229,41.997339)
)AS distance;

3. WITH points as
(
    select
        st_point(-93.610065,41.948637) AS A,
        st_point(-93.610229,41.997339) AS B,
        st_point(-93.619738,41.972017) AS C
),
 line as
 (
    select st_linestring(array[A,B]) as line
    from points
 )

 select
    st_distance(c,line) as distance_line
    from line,points;

4. WITH points as
(
    select
        st_point(-93.610065,41.948637) AS A,
        st_point(-93.610229,41.997339) AS B,
        st_point(-93.619738,41.972017) AS C
),
 line as
 (
    select st_linestring(array[A,B]) as line
    from points
 )

 select
    st_distance(c,line)*364567 as distance_line
    from line,points;


# Activity 3
1. Find distance from each point to all line strings in the database

```
select *, st_distance(
                st_point(a.longitude, a.latitude),
                st_linestring(array[
                        st_point(b.segment_start_longitude, b.segment_start_latitude),
                        st_point(b.segment_end_longitude,b.segment_end_latitude)])) distance from "raghu_gps_data"  a
cross join (select * from "aiml_class"."raghu_lrs_data" where not segment_start_latitude = segment_end_latitude and not segment_start_longitude = segment_end_longitude)  as b
where a.day =1 and a.hour = 1
```
2. Filter the nearest one for each point through a nested query

```
WITH filtered_gps AS (
    SELECT
        datapointid,
        journeyid,
        latitude AS gps_lat,
        longitude AS gps_lon,
        day
    FROM
        sanjita_gps_data
    WHERE
        day = 1 AND  -- Use AND instead of a comma
        hour = 1     -- Filter for hour = 1 as well
),
distance_calculation AS (
    SELECT
        gps.datapointid,
        gps.journeyid,
        gps.gps_lat,
        gps.gps_lon,
        lrs.route_id,
        lrs.segment_start_latitude AS start_lat,
        lrs.segment_start_longitude AS start_lon,
        lrs.segment_end_latitude AS end_lat,
        lrs.segment_end_longitude AS end_lon,
        -- Haversine formula for distance to start of segment
        6371 * acos(
            cos(radians(gps.gps_lat)) * cos(radians(lrs.segment_start_latitude)) *
            cos(radians(lrs.segment_start_longitude) - radians(gps.gps_lon)) +
            sin(radians(gps.gps_lat)) * sin(radians(lrs.segment_start_latitude))
        ) AS distance_to_start,
        -- Haversine formula for distance to end of segment
        6371 * acos(
            cos(radians(gps.gps_lat)) * cos(radians(lrs.segment_end_latitude)) *
            cos(radians(lrs.segment_end_longitude) - radians(gps.gps_lon)) +
            sin(radians(gps.gps_lat)) * sin(radians(lrs.segment_end_latitude))
        ) AS distance_to_end
    FROM
        filtered_gps gps
    CROSS JOIN
        sanjita_lrs_data lrs
)
SELECT * FROM distance_calculation;
```